In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from keras import Model, layers
from pathlib import Path
os.environ["KERAS_BACKEND"] = 'tensorflow'

In [14]:
df = pd.read_csv('data/cleaned_joined_data.csv')
df = df.set_index('Sample Name')
df

,cis-Nerolidol,trans-Nerolidol,trans-Nerolidol 1,trans-Nerolidol 2,trans-Ocimene,3-Carene,Camphene,Caryophyllene Oxide,Eucalyptol,Geraniol,...,fibromyalgia,crohn's_disease,phantom_limb_pain,epilepsy,multiple_sclerosis,parkinson's,tourette's_syndrome,alzheimer's,hiv/aids,tinnitus
Sample Name,,,,,,,,,,,,,,,,,,,,,
22,0.0,0.0,0.0,0.0,0.0,0.00,0.040000,0.000000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24k Gold,0.0,0.0,0.0,0.0,0.0,0.00,0.011000,0.003000,0.000,0.001000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3 Kings,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.533333,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3X Crazy,0.0,0.0,0.0,0.0,0.0,0.00,0.003000,0.004000,0.000,0.002000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
501st OG,0.0,0.0,0.0,0.0,0.0,0.00,0.006636,0.057818,0.000,0.000909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zelly’s Gift,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zeus OG,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zkittlez,0.0,0.0,0.0,0.0,0.0,0.00,0.010000,0.134000,0.001,0.011500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
fill_ins = {'Animal OG': 'Indica',
 'Cheesecake': 'Indica',
 'Dragon Lady': 'Sativa',
 'Green Marvel': 'Sativa',
 'Jack Straw': 'Sativa',
 'Lemon Punch': 'Hybrid',
 'Lychee': 'Hybrid',
 'M4': 'Hybrid',
 'Melon Cookies': 'Indica',
 'Mendo Purple': 'Hybrid',
 'Petroleum Nightmare': 'Hybrid',
 'Purgatory': 'Indica',
 'Silver Fox': 'Sativa',
 'Sour Walker': 'Hybrid',
 'Spirit In The Sky': 'Hybrid',
 'Strawberry Trainwreck': 'Sativa',
 'Tigers Blood': 'Indica',
 'Zprite': 'Hybrid'}
for strain in fill_ins:
    df.loc[strain, 'type'] = fill_ins[strain]

In [16]:
df.loc['Animal OG','type']

'Indica'

In [17]:
type_enc = OrdinalEncoder(categories = [['Indica', 'Hybrid', 'Sativa']], encoded_missing_value=-1, handle_unknown='use_encoded_value', unknown_value=-1)

In [18]:
df['encoded_type'] = type_enc.fit_transform(df['type'].values.reshape(-1,1))
df

,cis-Nerolidol,trans-Nerolidol,trans-Nerolidol 1,trans-Nerolidol 2,trans-Ocimene,3-Carene,Camphene,Caryophyllene Oxide,Eucalyptol,Geraniol,...,crohn's_disease,phantom_limb_pain,epilepsy,multiple_sclerosis,parkinson's,tourette's_syndrome,alzheimer's,hiv/aids,tinnitus,encoded_type
Sample Name,,,,,,,,,,,,,,,,,,,,,
22,0.0,0.0,0.0,0.0,0.0,0.00,0.040000,0.000000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24k Gold,0.0,0.0,0.0,0.0,0.0,0.00,0.011000,0.003000,0.000,0.001000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3 Kings,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.533333,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3X Crazy,0.0,0.0,0.0,0.0,0.0,0.00,0.003000,0.004000,0.000,0.002000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
501st OG,0.0,0.0,0.0,0.0,0.0,0.00,0.006636,0.057818,0.000,0.000909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zelly’s Gift,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
Zeus OG,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Zkittlez,0.0,0.0,0.0,0.0,0.0,0.00,0.010000,0.134000,0.001,0.011500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
predictors = ['encoded_type',
 'cis-Nerolidol',
 'trans-Nerolidol',
 'trans-Nerolidol 1',
 'trans-Nerolidol 2',
 'trans-Ocimene',
 '3-Carene',
 'Camphene',
 'Caryophyllene Oxide',
 'Eucalyptol',
 'Geraniol',
 'Guaiol',
 'Isopulegol',
 'Linalool',
 'Ocimene',
 'Terpinolene',
 'alpha-Bisabolol',
 'alpha-Humulene',
 'alpha-Pinene',
 'alpha-Terpinene',
 'beta-Caryophyllene',
 'beta-Myrcene',
 'beta-Ocimene',
 'beta-Pinene',
 'delta-Limonene',
 'gamma-Terpinene',
 'p-Cymene',
 'delta-9 THC-A',
 'delta-9 THC',
 'delta-8 THC',
 'THC-A',
 'THCV',
 'CBN',
 'CBD-A',
 'CBD',
 'CBDV',
 'CBDV-A',
 'delta-9 CBG-A',
 'delta-9 CBG',
 'CBC',
]
effects = ['relaxed',
 'happy',
 'euphoric',
 'uplifted',
 'sleepy',
 'dry_mouth',
 'dry_eyes',
 'dizzy',
 'paranoid',
 'anxious',
 'hungry',
 'talkative',
 'creative',
 'energetic',
 'focused',
 'giggly',
 'tingly',
 'aroused',]
illnesses = [ 
 'stress',
 'pain',
 'depression',
 'anxiety',
 'insomnia',
 'headache',
 'ptsd',
 'fatigue',
 'lack_of_appetite',
 'nausea',
 'headaches',
 'bipolar_disorder',
 'cancer',
 'cramps',
 'gastrointestinal_disorder',
 'inflammation',
 'muscle_spasms',
 'eye_pressure',
 'migraines',
 'asthma',
#  'anorexia',
 'arthritis',
 'add/adhd',
#  'muscular_dystrophy',
#  'hypertension',
 'glaucoma',
 'pms',
 'seizures',
 'spasticity',
#  'spinal_cord_injury',
 'fibromyalgia',
 "crohn's_disease",
#  'phantom_limb_pain',
 'epilepsy',
 'multiple_sclerosis',
 "parkinson's",
#  "tourette's_syndrome",
#  "alzheimer's",
 'hiv/aids',
#  'tinnitus'
 ]

In [22]:
df[effects].describe()

,stress,pain,depression,anxiety,insomnia,headache,ptsd,fatigue,lack_of_appetite,nausea,...,fibromyalgia,crohn's_disease,phantom_limb_pain,epilepsy,multiple_sclerosis,parkinson's,tourette's_syndrome,alzheimer's,hiv/aids,tinnitus
count,1111.000000,1111.000000,1111.000000,1111.000000,1111.000000,1111.000000,1111.000000,1111.000000,1111.000000,1111.000000,...,1111.000000,1111.000000,1111.0,1111.000000,1111.000000,1111.000000,1111.0,1111.0,1111.000000,1111.0
mean,0.295500,0.222367,0.212394,0.213258,0.105374,0.019523,0.010423,0.041089,0.027741,0.014365,...,0.000207,0.000297,0.0,0.000594,0.000297,0.000297,0.0,0.0,0.000450,0.0
std,0.161426,0.160341,0.140200,0.143588,0.145258,0.036156,0.052308,0.086129,0.086484,0.068227,...,0.005238,0.009900,0.0,0.015746,0.009900,0.009900,0.0,0.0,0.015001,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
25%,0.230000,0.120000,0.160000,0.135000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
50%,0.330000,0.240000,0.230000,0.240000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
75%,0.390000,0.320000,0.280000,0.300000,0.210000,0.030000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.500000,0.500000,1.000000,1.000000,1.000000,...,0.160000,0.330000,0.0,0.500000,0.330000,0.330000,0.0,0.0,0.500000,0.0


In [21]:
df[effects].head()

,relaxed,happy,euphoric,uplifted,sleepy,dry_mouth,dry_eyes,dizzy,paranoid,anxious,hungry,talkative,creative,energetic,focused,giggly,tingly,aroused
Sample Name,,,,,,,,,,,,,,,,,,
22,0.00,0.50,0.50,0.50,0.00,0.10,0.00,0.00,0.00,0.10,0.0,0.0,0.00,0.5,0.5,0.0,0.0,0.0
24k Gold,0.61,0.55,0.41,0.44,0.00,0.23,0.11,0.05,0.00,0.02,0.0,0.0,0.24,0.0,0.0,0.0,0.0,0.0
3 Kings,0.58,0.58,0.48,0.48,0.00,0.21,0.13,0.05,0.02,0.02,0.0,0.0,0.31,0.0,0.0,0.0,0.0,0.0
3X Crazy,0.76,0.48,0.47,0.31,0.38,0.23,0.16,0.03,0.02,0.04,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
501st OG,0.81,0.47,0.44,0.25,0.41,0.19,0.06,0.01,0.00,0.01,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [33]:
df[effects].sum()

relaxed      521.95
happy        526.39
euphoric     425.44
uplifted     392.07
sleepy       137.64
dry_mouth    236.68
dry_eyes     132.82
dizzy         49.84
paranoid      36.04
anxious       33.48
hungry        65.26
talkative     29.68
creative     124.35
energetic    116.50
focused       72.68
giggly        22.42
tingly        23.09
aroused        6.21
dtype: float64

In [30]:
# drop any columns with all zeros
df[illnesses].sum()==0

stress                       False
pain                         False
depression                   False
anxiety                      False
insomnia                     False
headache                     False
ptsd                         False
fatigue                      False
lack_of_appetite             False
nausea                       False
headaches                    False
bipolar_disorder             False
cancer                       False
cramps                       False
gastrointestinal_disorder    False
inflammation                 False
muscle_spasms                False
eye_pressure                 False
migraines                    False
asthma                       False
anorexia                      True
arthritis                    False
add/adhd                     False
muscular_dystrophy            True
hypertension                  True
glaucoma                     False
pms                          False
seizures                     False
spasticity          

In [34]:
# empty_cols = ['anorexia',
#               'muscular_dystrophy',
#               'hypertension',
#               'spinal_cord_injury',
#               'phantom_limb_pain',
#               "tourette's_syndrome",
#               "alzheimer's",
#               'tinnitus'
#               ]
# df = df.drop(columns=empty_cols)
# df

,cis-Nerolidol,trans-Nerolidol,trans-Nerolidol 1,trans-Nerolidol 2,trans-Ocimene,3-Carene,Camphene,Caryophyllene Oxide,Eucalyptol,Geraniol,...,pms,seizures,spasticity,fibromyalgia,crohn's_disease,epilepsy,multiple_sclerosis,parkinson's,hiv/aids,encoded_type
Sample Name,,,,,,,,,,,,,,,,,,,,,
22,0.0,0.0,0.0,0.0,0.0,0.00,0.040000,0.000000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24k Gold,0.0,0.0,0.0,0.0,0.0,0.00,0.011000,0.003000,0.000,0.001000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3 Kings,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.533333,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3X Crazy,0.0,0.0,0.0,0.0,0.0,0.00,0.003000,0.004000,0.000,0.002000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
501st OG,0.0,0.0,0.0,0.0,0.0,0.00,0.006636,0.057818,0.000,0.000909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zelly’s Gift,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
Zeus OG,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Zkittlez,0.0,0.0,0.0,0.0,0.0,0.00,0.010000,0.134000,0.001,0.011500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
df_binary_target = df.copy()
# for each column
for col in df[effects+illnesses].columns:
    print(col)
    # calculate the mean
    col_mean = df[col].mean()
    # for each row
    for row in df.index:
        # if gte to column mean
        if df.loc[row,col]>=col_mean:
            # set to true
            df_binary_target.loc[row,col] = 1
        else:
            df_binary_target.loc[row,col] = 0
df_binary_target

relaxed
happy
euphoric
uplifted
sleepy
dry_mouth
dry_eyes
dizzy
paranoid
anxious
hungry
talkative
creative
energetic
focused
giggly
tingly
aroused
stress
pain
depression
anxiety
insomnia
headache
ptsd
fatigue
lack_of_appetite
nausea
headaches
bipolar_disorder
cancer
cramps
gastrointestinal_disorder
inflammation
muscle_spasms
eye_pressure
migraines
asthma
arthritis
add/adhd
glaucoma
pms
seizures
spasticity
fibromyalgia
crohn's_disease
epilepsy
multiple_sclerosis
parkinson's
hiv/aids


,cis-Nerolidol,trans-Nerolidol,trans-Nerolidol 1,trans-Nerolidol 2,trans-Ocimene,3-Carene,Camphene,Caryophyllene Oxide,Eucalyptol,Geraniol,...,pms,seizures,spasticity,fibromyalgia,crohn's_disease,epilepsy,multiple_sclerosis,parkinson's,hiv/aids,encoded_type
Sample Name,,,,,,,,,,,,,,,,,,,,,
22,0.0,0.0,0.0,0.0,0.0,0.00,0.040000,0.000000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24k Gold,0.0,0.0,0.0,0.0,0.0,0.00,0.011000,0.003000,0.000,0.001000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3 Kings,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.533333,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3X Crazy,0.0,0.0,0.0,0.0,0.0,0.00,0.003000,0.004000,0.000,0.002000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
501st OG,0.0,0.0,0.0,0.0,0.0,0.00,0.006636,0.057818,0.000,0.000909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zelly’s Gift,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
Zeus OG,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Zkittlez,0.0,0.0,0.0,0.0,0.0,0.00,0.010000,0.134000,0.001,0.011500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
# df_binary_target[effects].apply(,axis='columns')

In [48]:
X = df_binary_target[predictors]
y_effects = df_binary_target[effects]
y_illnesses = df_binary_target[illnesses]

X_train, X_test, y_train_effects, y_test_effects, y_train_illnesses, y_test_illnesses = train_test_split(X, y_effects, y_illnesses)

In [51]:
X_train

,encoded_type,cis-Nerolidol,trans-Nerolidol,trans-Nerolidol 1,trans-Nerolidol 2,trans-Ocimene,3-Carene,Camphene,Caryophyllene Oxide,Eucalyptol,...,THC-A,THCV,CBN,CBD-A,CBD,CBDV,CBDV-A,delta-9 CBG-A,delta-9 CBG,CBC
Sample Name,,,,,,,,,,,,,,,,,,,,,
Venom OG,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.011,0.111714,0.000,...,17.680000,0.000000,0.000000,0.057143,0.018571,0.0,0.00000,0.731429,0.067143,0.015714
American Pie,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.420000,0.480000,0.0,0.00000,0.000000,0.510000,0.020000
Aurora Indica,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.250000,0.000,...,0.000000,0.000000,0.000000,0.170000,0.230000,0.0,0.00000,0.920000,0.000000,0.070000
Zkittlez,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.010,0.134000,0.001,...,12.720000,0.065000,0.370000,0.065000,0.000000,0.0,0.00000,0.720000,0.460000,0.355000
Dr. Grinspoon,2.0,0.0,0.0,0.0,0.0,0.0,0.010000,0.000,0.205000,0.000,...,0.000000,0.010000,0.015000,0.085000,0.055000,0.0,0.00000,0.095000,0.460000,0.060000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Space Dawg,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.373636,0.000,...,0.000000,0.000000,0.000000,0.125455,0.132727,0.0,0.00000,0.514545,0.212727,0.020000
Elephant,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.005,0.098000,0.000,...,9.664000,0.001000,0.001000,0.098000,0.117000,0.0,0.00000,0.553000,0.001000,0.038000
Critical Kush,0.0,0.0,0.0,0.0,0.0,0.0,0.001478,0.000,0.167000,0.000,...,1.886957,0.007391,0.010870,0.104783,0.102174,0.0,0.00087,0.768696,0.106957,0.055652


In [49]:
# Scale the X data by using StandardScaler()
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [50]:
X_train_scaled.shape, y_train_effects.shape

((833, 40), (833, 18))

In [53]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_effects_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=128,
        step=2), activation=activation, input_dim=len(X.columns)))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 16)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=128,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=y_train_effects.shape[1], activation="softmax"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [54]:
tuner_effects = kt.Hyperband(
    create_effects_model,
    objective="val_accuracy",
    max_epochs=100,
    hyperband_iterations=2,
    overwrite=True)

/Users/justified/.pyenv/versions/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [55]:
# Run the kerastuner search for best hyperparameters
tuner_effects.search(X_train_scaled,y_train_effects,epochs=20,validation_data=(X_test_scaled,y_test_effects))

Trial 497 Complete [00h 00m 12s]
val_accuracy: 0.564748227596283

Best val_accuracy So Far: 0.7374100685119629
Total elapsed time: 00h 41m 40s


In [56]:
# Get best model hyperparameters
best_hyper_effects = tuner_effects.get_best_hyperparameters(1)[0]
best_hyper_effects.values

{'activation': 'relu',
 'first_units': 103,
 'num_layers': 12,
 'units_0': 21,
 'units_1': 103,
 'units_2': 95,
 'units_3': 3,
 'units_4': 41,
 'units_5': 115,
 'units_6': 123,
 'units_7': 37,
 'units_8': 61,
 'units_9': 31,
 'units_10': 47,
 'units_11': 13,
 'units_12': 85,
 'units_13': 67,
 'units_14': 81,
 'units_15': 105,
 'tuner/epochs': 34,
 'tuner/initial_epoch': 12,
 'tuner/bracket': 4,
 'tuner/round': 3,
 'tuner/trial_id': '0130'}

In [59]:
# Evaluate best model against full test data
best_model = tuner_effects.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test,y_test_effects,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

/Users/justified/.pyenv/versions/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/justified/.pyenv/versions/3.12.1/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 58 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


9/9 - 0s - 14ms/step - accuracy: 0.6906 - loss: 0.5477
Loss: 0.5477299690246582, Accuracy: 0.6906474828720093


In [60]:
file_path_effects_model_2 = Path('effects-model-2.keras')
best_model.save(file_path_effects_model_2)